In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import os

In [3]:
path = '/.../Time-series processed data'
os.chdir(path)

In [5]:
ftm0_data = pd.read_csv('ftm_processed_data_nobody.csv')

In [6]:
ftm1_data = pd.read_csv('ftm_processed_data_1person.csv')

In [7]:
ftm2_data = pd.read_csv('ftm_processed_data_2person.csv')

In [12]:
ftm0_data = ftm0_data.reset_index()

In [13]:
ftm1_data = ftm1_data.reset_index()

In [14]:
ftm2_data = ftm2_data.reset_index()

In [15]:
ftm0_data['label'] = '0'

In [16]:
ftm1_data['label'] = '1'

In [17]:
ftm2_data['label'] = '2'

In [32]:
ftm_data = pd.concat([ftm0_data, ftm1_data, ftm2_data]).reset_index()

## Feature extraction

#### Aggregation over channels

In [34]:
def ch_aggregation(df, cols, metrics):
    df_features = df.copy()
    for m in metrics:
        col_name=''
        for col in cols: 
            col_name = col_name + col + '_'
            #col_name = col_name + col[-1]
        col_name = col_name + m
        if m == 'mean':
            df_features[col_name] = df_features[cols].mean(axis=1)
        if m == 'max':
            df_features[col_name] = df_features[cols].max(axis=1)
        if m == 'min':
            df_features[col_name] = df_features[cols].min(axis=1)
        if m == 'std':
            df_features[col_name] = df_features[cols].std(axis=1)
    return(df_features)
            

In [35]:
ftm_ch_features = ftm_data.copy()
for i in range(0,8):
    ftm_ch_features = ch_aggregation(ftm_ch_features, cols=['ch'+str(i+1),'ch'+str(i+2),'ch'+str(i+3)], metrics=['mean','max','min','std'])
 

In [36]:
ftm_ch_features.columns

Index(['level_0', 'index', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7',
       'ch8', 'ch9', 'ch10', 'label', 'ch1_ch2_ch3_mean', 'ch1_ch2_ch3_max',
       'ch1_ch2_ch3_min', 'ch1_ch2_ch3_std', 'ch2_ch3_ch4_mean',
       'ch2_ch3_ch4_max', 'ch2_ch3_ch4_min', 'ch2_ch3_ch4_std',
       'ch3_ch4_ch5_mean', 'ch3_ch4_ch5_max', 'ch3_ch4_ch5_min',
       'ch3_ch4_ch5_std', 'ch4_ch5_ch6_mean', 'ch4_ch5_ch6_max',
       'ch4_ch5_ch6_min', 'ch4_ch5_ch6_std', 'ch5_ch6_ch7_mean',
       'ch5_ch6_ch7_max', 'ch5_ch6_ch7_min', 'ch5_ch6_ch7_std',
       'ch6_ch7_ch8_mean', 'ch6_ch7_ch8_max', 'ch6_ch7_ch8_min',
       'ch6_ch7_ch8_std', 'ch7_ch8_ch9_mean', 'ch7_ch8_ch9_max',
       'ch7_ch8_ch9_min', 'ch7_ch8_ch9_std', 'ch8_ch9_ch10_mean',
       'ch8_ch9_ch10_max', 'ch8_ch9_ch10_min', 'ch8_ch9_ch10_std'],
      dtype='object')

In [37]:
ftm_ch_features = ftm_ch_features.drop(['level_0', 'index'], axis=1)

In [38]:
# ftm_ch_features.to_csv('labeled_ftm channel features_data.csv')

#### Aggregation over time using sliding windows (window size = 10 second/5 samples)

In [24]:
def rme(x):
    return np.sqrt(sum(x**2)/len(x))

In [122]:
def ind_aggregation(df):
    return df.groupby(['win']).agg({'ch1':['mean', 'std', 'median', rme], 
                                    'ch2':['mean', 'std', 'median', rme],
                                    'ch3':['mean', 'std', 'median', rme], 
                                    'ch4':['mean', 'std', 'median', rme], 
                                    'ch5':['mean', 'std', 'median', rme], 
                                    'ch6':['mean', 'std', 'median', rme], 
                                    'ch7':['mean', 'std', 'median', rme], 
                                    'ch8':['mean', 'std', 'median', rme], 
                                    'ch9':['mean', 'std', 'median', rme], 
                                    'ch10':['mean', 'std', 'median', rme]}).reset_index()

In [118]:
def sliding_win(df, win_size):
    sliding_data = pd.DataFrame()
    for i in range(len(df)-(win_size-1)):
        window_data = df[i:i+win_size]
        window_data['win'] = i
        sliding_data =sliding_data.append(window_data)
    return sliding_data   

In [131]:
ftm0_sliding_ind_features = ind_aggregation(sliding_win(ftm0_data,5)).drop(['win'], axis=1)
ftm0_sliding_ind_features.columns = ["".join(x) for x in ftm0_sliding_ind_features.columns.ravel()]
ftm0_sliding_ind_features['label'] = 0

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3301846962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  window_data['win'] = i
/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3369772976.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm0_sliding_ind_features.columns = ["".join(x) for x in ftm0_sliding_ind_features.columns.ravel()]


In [132]:
ftm1_sliding_ind_features = ind_aggregation(sliding_win(ftm1_data,5)).drop(['win'], axis=1)
ftm1_sliding_ind_features.columns = ["".join(x) for x in ftm1_sliding_ind_features.columns.ravel()]
ftm1_sliding_ind_features['label'] = 1

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3301846962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  window_data['win'] = i
/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3856569549.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm1_sliding_ind_features.columns = ["".join(x) for x in ftm1_sliding_ind_features.columns.ravel()]


In [133]:
ftm2_sliding_ind_features = ind_aggregation(sliding_win(ftm2_data,5)).drop(['win'], axis=1)
ftm2_sliding_ind_features.columns = ["".join(x) for x in ftm2_sliding_ind_features.columns.ravel()]
ftm2_sliding_ind_features['label'] = 2

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3301846962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  window_data['win'] = i
/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/667353732.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm2_sliding_ind_features.columns = ["".join(x) for x in ftm2_sliding_ind_features.columns.ravel()]


In [137]:
# ftm0_sliding_ind_features.to_csv('labeled_ftm0 index features_sliding win5_data.csv')

In [138]:
# ftm1_sliding_ind_features.to_csv('labeled_ftm1 index features_sliding win5_data.csv')

In [139]:
# ftm2_sliding_ind_features.to_csv('labeled_ftm2 index features_sliding win5_data.csv')

In [135]:
ftm_sliding_ind_features = pd.concat([ftm0_sliding_ind_features, ftm1_sliding_ind_features, ftm2_sliding_ind_features]).reset_index()

In [136]:
# ftm_sliding_ind_features.to_csv('labeled_ftm index features_sliding win5_data.csv')

#### Aggregating over time using sequential windows (window size = 10 second/5 samples)

In [98]:
win_size = 5
for i in range(len(ftm0_data)-win_size):
    a = ftm0_data[i:i+win_size].agg({'ch1':['mean', 'std', 'median', rme], 'ch2':['mean', 'std', 'median', rme],'ch3':['mean', 'std', 'median', rme], 'ch4':['mean', 'std', 'median', rme], 'ch5':['mean', 'std', 'median', rme], 'ch6':['mean', 'std', 'median', rme], 'ch7':['mean', 'std', 'median', rme], 'ch8':['mean', 'std', 'median', rme], 'ch9':['mean', 'std', 'median', rme], 'ch10':['mean', 'std', 'median', rme]}).reset_index()
    ftm0_featur = ftm0_featur.append(a)


In [66]:
def ind_segmentation(win_size, df):
    df_seg = df.copy()
    df_seg['win']=0
    for w in range(int((len(df_seg)-win_size+1)/win_size)+1):
        df_seg.win[(w*win_size):(w*win_size)+win_size] = w
    return df_seg

In [67]:
ftm0_seg_data = ind_segmentation(5, ftm0_data)

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3631869467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg.win[(w*win_size):(w*win_size)+win_size] = w


In [45]:
ftm1_seg_data = ind_segmentation(5, ftm1_data)

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3631869467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg.win[(w*win_size):(w*win_size)+win_size] = w


In [46]:
ftm2_seg_data = ind_segmentation(5, ftm2_data)

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/3631869467.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg.win[(w*win_size):(w*win_size)+win_size] = w


In [83]:
ftm0_ind_features = ind_aggregation(ftm0_seg_data).drop(['win'], axis=1)
ftm0_ind_features.columns = ["".join(x) for x in ftm0_ind_features.columns.ravel()]
ftm0_ind_features['label'] = 0

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/1330315466.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm0_ind_features.columns = ["".join(x) for x in ftm0_ind_features.columns.ravel()]


In [84]:
ftm0_ind_features

,ch1mean,ch1std,ch1median,ch1rme,ch2mean,ch2std,ch2median,ch2rme,ch3mean,ch3std,...,ch8rme,ch9mean,ch9std,ch9median,ch9rme,ch10mean,ch10std,ch10median,ch10rme,label
0,9.990,11.800983,4.68,14.533088,2.626,0.192821,2.60,2.631657,1.326,0.232551,...,3.744361,2.860,0.000000,2.86,2.860000,2.886,0.058138,2.86,2.886468,0
1,4.472,0.299282,4.68,4.480004,2.730,0.260000,2.73,2.739887,1.612,0.269574,...,3.667291,2.834,0.058138,2.86,2.834477,2.938,0.071204,2.99,2.938690,0
2,4.212,0.417217,4.42,4.228499,2.470,0.450333,2.34,2.502627,1.456,0.232551,...,3.718545,2.782,0.071204,2.73,2.782729,2.990,0.000000,2.99,2.990000,0
3,4.706,0.684200,4.55,4.745623,2.444,0.515102,2.73,2.487046,1.300,0.379012,...,3.698951,2.756,0.250060,2.86,2.765061,2.964,0.142408,2.86,2.966736,0
4,9.520,11.850859,4.55,14.247269,2.184,0.819103,2.60,2.303606,1.040,0.623458,...,3.593271,2.600,0.356020,2.86,2.619427,2.548,0.548471,2.86,2.594795,0
5,3.146,0.169499,3.12,3.149651,0.988,0.116276,1.04,0.993459,0.234,0.108766,...,3.380000,2.210,0.130000,2.21,2.213057,1.924,0.058138,1.95,1.924703,0
6,3.172,0.197155,3.25,3.176898,0.910,0.091924,0.91,0.913707,0.156,0.108766,...,3.250000,2.184,0.058138,2.21,2.184619,1.976,0.108766,1.95,1.978393,0
7,3.172,0.071204,3.12,3.172639,0.962,0.071204,0.91,0.964106,0.104,0.142408,...,3.328609,2.158,0.071204,2.21,2.158940,2.054,0.108766,2.08,2.056303,0
8,4.914,0.481176,4.81,4.932811,1.898,0.071204,1.95,1.899068,1.066,0.425241,...,3.355411,2.262,0.116276,2.34,2.264390,2.080,0.091924,2.08,2.081624,0
9,9.024,11.905540,3.77,13.958011,2.132,0.721471,2.08,2.227519,0.468,0.465102,...,3.254157,2.184,0.192821,2.21,2.190799,2.106,0.108766,2.08,2.108246,0


In [85]:
ftm1_ind_features = ind_aggregation(ftm1_seg_data).drop(['win'], axis=1)
ftm1_ind_features.columns = ["".join(x) for x in ftm1_ind_features.columns.ravel()]
ftm1_ind_features['label'] = 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/1421873090.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm1_ind_features.columns = ["".join(x) for x in ftm1_ind_features.columns.ravel()]


In [86]:
ftm2_ind_features = ind_aggregation(ftm2_seg_data).drop(['win'], axis=1)
ftm2_ind_features.columns = ["".join(x) for x in ftm2_ind_features.columns.ravel()]
ftm2_ind_features['label'] = 2

/var/folders/66/mfk28_pn1934k74x619jkgz40000gn/T/ipykernel_8824/2703145185.py:2: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  ftm2_ind_features.columns = ["".join(x) for x in ftm2_ind_features.columns.ravel()]


In [88]:
# ftm0_ind_features.to_csv('labeled_ftm0 index features_win5_data.csv')

In [89]:
# ftm1_ind_features.to_csv('labeled_ftm1 index features_win5_data.csv')

In [90]:
# ftm2_ind_features.to_csv('labeled_ftm2 index features_win5_data.csv')

In [91]:
ftm_ind_features = pd.concat([ftm0_ind_features, ftm1_ind_features, ftm2_ind_features]).reset_index()

In [92]:
# ftm_ind_features.to_csv('labeled_ftm index features_win5_data.csv')